# GiftMuse Atelier Concierge - Gemini Colab App

Everything required to demo the GiftMuse Atelier gifting concierge in Google Colab lives in this notebook. It bundles a Gemini-powered agent, lead & feedback logging, sample interactions, and a colorful Gradio chat UI.

## Quick Start
1. Place your `.env` file alongside this notebook (already populated per your request).
2. Open the notebook in Colab.
3. (Optional) Run the install cell if the required packages are missing.
4. Execute the remaining cells sequentially to chat with the concierge and view logged leads/feedback.

In [ ]:
# Optional: install dependencies if your environment does not already have them.
# Uncomment if needed.
# !pip install -q google-generativeai gradio PyPDF2 python-dotenv

## Load Gemini API Key from `.env`
Your key is loaded securely via `python-dotenv` so it is not hard-coded in the notebook.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
if not GEMINI_API_KEY:
    raise ValueError('GEMINI_API_KEY is missing. Add it to your .env file before proceeding.')
GEMINI_API_KEY

## Preferred Gemini Models
The agent will default to `models/gemini-2.5-flash` and automatically fall back to additional options if needed.

In [ ]:
MODEL_CANDIDATES = [
    'models/gemini-2.5-flash',
    'models/gemini-1.5-flash-latest',
    'models/gemini-1.5-flash',
    'models/gemini-1.5-flash-8b-latest',
    'models/gemini-1.5-pro-latest',
    'models/gemini-1.5-pro',
]
MODEL_CANDIDATES

In [ ]:
from __future__ import annotations

import json
from datetime import datetime, timezone
from pathlib import Path
from typing import Dict, List, Optional

import google.generativeai as genai
from google.generativeai.types import GenerationConfig
from google.api_core import exceptions as google_exceptions

# --- Business content -------------------------------------------------------
summary_text = "GiftMuse Atelier is a gifting intelligence studio that blends warm human curation with agile AI research. We deliver quick, heartfelt gift ideas that feel tailor-made for every person and milestone.\n\nWe focus on concierge gifting paths: on-demand scouting sprints, event gifting programs, and the GiftGlow Corporate service for sales and HR teams. Clients trust our 120 artisan maker partners and sustainability pledge donating one percent of concierge packages to creativity grants."
profile_snippet = "GiftMuse Atelier Business Snapshot\nMission: Replace gifting stress with confidence through curated, sentimental suggestions.\nServices: Gift scouting sprints, signature event gifting programs, GiftGlow Corporate concierge.\nTeam: Ava Moreno (Founder), Idris Patel (CTO), Priya Das (Customer Journey Lead).\nProcess: Blend human curiosity with AI taste modeling to spotlight artisan and sustainable vendors.\nValue: Collective of 120 makers, sustainability pledge, wrap-and-delivery partners.\nTone: Warm, organized, proactive about transforming occasions into memories."

# --- Data directory for logs ------------------------------------------------
DATA_DIR = Path("giftmuse_data")
DATA_DIR.mkdir(exist_ok=True)
LEADS_LOG = DATA_DIR / "leads.jsonl"
FEEDBACK_LOG = DATA_DIR / "feedback.jsonl"

# --- Tool helpers -----------------------------------------------------------
def _timestamp() -> str:
    return datetime.now(timezone.utc).isoformat()


def _write_log(path: Path, payload: Dict[str, str]) -> None:
    with path.open("a", encoding="utf-8") as handle:
        handle.write(json.dumps(payload, ensure_ascii=False))
        handle.write("\n")


def record_customer_interest(email: str = "", name: str = "", message: Optional[str] = None) -> str:
    entry = {
        "type": "lead",
        "timestamp": _timestamp(),
        "name": (name or "").strip(),
        "email": (email or "").strip().lower(),
        "message": (message or "").strip(),
    }
    _write_log(LEADS_LOG, entry)
    if entry["name"] or entry["email"]:
        return "Thanks! I captured your details so our concierge can reach out soon."
    return "Appreciate the interest. I logged the note for our concierge team."


def record_feedback(question: str) -> str:
    entry = {
        "type": "feedback",
        "timestamp": _timestamp(),
        "question": (question or "").strip(),
    }
    _write_log(FEEDBACK_LOG, entry)
    return "I saved that question for the GiftMuse Atelier team so we can follow up promptly."


GEMINI_FUNCTION_DECLARATIONS = [
    {
        "name": "record_customer_interest",
        "description": "Capture a lead's contact details and notes for concierge follow-up.",
        "parameters": {
            "type": "object",
            "properties": {
                "email": {"type": "string", "description": "Customer email address, if supplied."},
                "name": {"type": "string", "description": "Customer name or representative."},
                "message": {"type": "string", "description": "Any context about their gifting needs or requests."},
            },
            "required": [],
        },
    },
    {
        "name": "record_feedback",
        "description": "Log unanswered questions or feedback for the GiftMuse Atelier team.",
        "parameters": {
            "type": "object",
            "properties": {
                "question": {"type": "string", "description": "The feedback item or question we could not answer."},
            },
            "required": ["question"],
        },
    },
]


# --- Agent utilities -------------------------------------------------------
def build_system_prompt(summary: str, profile: str) -> str:
    return (
        "You are the concierge for GiftMuse Atelier, a joyful gifting intelligence studio. "
        "Reply with upbeat, organized guidance grounded in the business profile. "
        "Suggest creative gift paths quickly, highlight artisan or sustainable makers, and invite clients to share contact details. "
        "If you lack information, apologize briefly and call the feedback tool so the team can follow up.\n\n"
        f"Business summary:\n{summary}\n\nSnapshot from the detailed profile:\n{profile}\n"
    )


class GiftMuseAgent:
    def __init__(
        self,
        api_key: str,
        model_candidates: Optional[List[str]] = None,
        system_prompt: Optional[str] = None,
    ) -> None:
        if not api_key:
            raise ValueError("Gemini API key is required.")
        genai.configure(api_key=api_key)

        defaults = [
            "models/gemini-2.5-flash",
            "models/gemini-1.5-flash-latest",
            "models/gemini-1.5-flash",
            "models/gemini-1.5-flash-8b-latest",
            "models/gemini-1.5-pro-latest",
            "models/gemini-1.5-pro",
        ]
        supplied = model_candidates or []
        self.model_candidates: List[str] = []
        for candidate in supplied + defaults:
            if candidate and candidate not in self.model_candidates:
                self.model_candidates.append(candidate)

        self.system_prompt = system_prompt or build_system_prompt(summary_text, profile_snippet)
        self._model = None
        self._chat = None
        self._active_index = 0
        self._init_chat(start_index=0)

    def _init_chat(self, start_index: int) -> None:
        last_error: Optional[Exception] = None
        for idx in range(start_index, len(self.model_candidates)):
            candidate = self.model_candidates[idx]
            try:
                model = genai.GenerativeModel(
                    model_name=candidate,
                    system_instruction=self.system_prompt,
                    tools=[{"function_declarations": GEMINI_FUNCTION_DECLARATIONS}],
                )
                chat = model.start_chat(history=[])
                self._model = model
                self._chat = chat
                self._active_index = idx
                self.model_name = candidate
                if idx != start_index:
                    print(f"Switched to Gemini model: {candidate}")
                else:
                    print(f"Using Gemini model: {candidate}")
                return
            except (
                google_exceptions.NotFound,
                google_exceptions.FailedPrecondition,
                google_exceptions.PermissionDenied,
            ) as exc:
                last_error = exc
                continue

        raise RuntimeError(
            "Unable to initialize a Gemini model. "
            f"Tried: {', '.join(self.model_candidates)}. Last error: {last_error}"
        )

    def _rotate_model(self) -> None:
        next_index = self._active_index + 1
        if next_index >= len(self.model_candidates):
            raise RuntimeError(
                "All configured Gemini models returned errors. "
                f"Models tried: {', '.join(self.model_candidates)}"
            )
        self._init_chat(start_index=next_index)

    @staticmethod
    def _extract_text(response) -> str:
        for candidate in response.candidates:
            parts = []
            for part in candidate.content.parts:
                text = getattr(part, "text", None)
                if text:
                    parts.append(text)
            if parts:
                return "\n".join(parts).strip()
        return ""

    @staticmethod
    def _next_tool_call(response) -> Optional[Dict[str, object]]:
        for candidate in response.candidates:
            for part in candidate.content.parts:
                call = getattr(part, "function_call", None)
                if call:
                    return {"name": call.name, "args": dict(call.args or {})}
        return None

    @staticmethod
    def _invoke_tool(name: str, arguments: Dict[str, object]) -> Dict[str, object]:
        if name == "record_customer_interest":
            result_text = record_customer_interest(
                email=arguments.get("email", ""),
                name=arguments.get("name", ""),
                message=arguments.get("message"),
            )
        elif name == "record_feedback":
            result_text = record_feedback(question=arguments.get("question", ""))
        else:
            result_text = "Tool not implemented."
        return {
            "function_response": {
                "name": name,
                "response": {"text": result_text},
            }
        }

    def respond(self, user_input: str, temperature: float = 0.6) -> str:
        attempts = 0
        while attempts < len(self.model_candidates):
            try:
                config = GenerationConfig(temperature=temperature)
                response = self._chat.send_message(user_input, generation_config=config)
                tool_call = self._next_tool_call(response)
                while tool_call:
                    tool_output = self._invoke_tool(tool_call["name"], tool_call["args"])
                    response = self._chat.send_message(tool_output, generation_config=config)
                    tool_call = self._next_tool_call(response)
                return self._extract_text(response)
            except (
                google_exceptions.NotFound,
                google_exceptions.FailedPrecondition,
                google_exceptions.PermissionDenied,
            ) as exc:
                attempts += 1
                if attempts >= len(self.model_candidates):
                    raise RuntimeError(
                        "All configured Gemini models returned errors during respond(). "
                        f"Models tried: {', '.join(self.model_candidates)}"
                    ) from exc
                print(f"{exc.__class__.__name__}: {exc} -- trying next model...")
                self._rotate_model()

        raise RuntimeError("Unable to obtain a response from any Gemini model.")

    def reset(self) -> None:
        self._chat = self._model.start_chat(history=[])


In [ ]:
SYSTEM_PROMPT = build_system_prompt(summary_text, profile_snippet)
print(f"System prompt characters: {len(SYSTEM_PROMPT)}")
agent = GiftMuseAgent(api_key=GEMINI_API_KEY, model_candidates=MODEL_CANDIDATES, system_prompt=SYSTEM_PROMPT)
print("Active Gemini model:", agent.model_name)


## Try the Concierge
Uncomment the cells below to sample the assistant and trigger tool logging.

In [ ]:
# agent.reset()
# reply = agent.respond("We are hosting a sunset engagement; can you suggest a dazzling keepsake?")
# print(reply)


In [ ]:
# reply = agent.respond("Capture my info: Layla Haddad, layla@example.com, 25 eco gift boxes for our partners.")
# print(reply)


## Launch Gradio Chat Interface

In [ ]:
import gradio as gr

gift_theme = gr.themes.Soft(
    primary_hue="pink",
    secondary_hue="rose",
    neutral_hue="gray",
    font=("Poppins", "sans-serif"),
)

GIFT_CSS = """
.gradio-container {background: linear-gradient(135deg, #fff5f7 0%, #ffeefc 100%);}
.chatbot .message.user {background: rgba(255, 182, 193, 0.25); border-radius: 18px;}
.chatbot .message.bot {background: rgba(255, 255, 255, 0.95); border-radius: 18px; border: 1px solid #ffd4e5;}
.gradio-container .title {font-weight: 700; letter-spacing: 0.02em;}
.gradio-container button {background: #ff5c8a !important; border-color: #ff5c8a !important;}
"""

def gradio_respond(message, history):
    if not history:
        agent.reset()
    return agent.respond(message)

chat = gr.ChatInterface(
    fn=gradio_respond,
    type="messages",
    title="GiftMuse Atelier Concierge",
    description="Share the occasion, vibe, and budget - I will suggest handcrafted gifting paths and tee up our concierge team.",
    theme=gift_theme,
    css=GIFT_CSS,
    examples=[
        "I need a wow-worthy gift for a 10-year work anniversary celebration.",
        "We are planning 30 sustainable welcome kits - can you recommend artisan makers?",
        "Do you offer last-minute curation and gift wrapping for Dubai deliveries?",
    ],
)

chat.launch(share=False)


## Review Logged Leads and Feedback

In [ ]:
for log_path in [LEADS_LOG, FEEDBACK_LOG]:
    print(f"\n{log_path.name}:")
    if log_path.exists() and log_path.read_text().strip():
        for line in log_path.read_text(encoding="utf-8").splitlines():
            print(json.loads(line))
    else:
        print("No entries yet.")
